# Milestone 05: 銀行帳戶系統 - 詳細需求規格

## 📋 專案概述

本專案要求您設計並實作一個完整的**銀行帳戶管理系統**，整合 Ch16-19 的物件導向程式設計概念。這是一個企業級的專案，將展示您對物件導向四大支柱（封裝、繼承、多型、抽象）的深度理解與實際應用能力。

### 🎯 整合技術要點
- **Ch16 類別與物件**：基礎類別設計、實例化、屬性與方法
- **Ch17 封裝與資訊隱藏**：私有屬性、property 裝飾器、getter/setter
- **Ch18 繼承與多型**：類別繼承、方法覆寫、多型實作
- **Ch19 特殊方法**：魔術方法、運算子重載、物件比較

---

## 🏗️ 系統架構設計（System Architecture）

### UML 類別圖（Class Diagram）

```mermaid
classDiagram
    class BankAccount {
        <<abstract>>
        -_account_number: str
        -_balance: float
        -_customer: Customer
        -_transactions: List[Transaction]
        -_created_date: datetime
        -_is_active: bool
        +balance: float
        +account_number: str
        +deposit(amount: float): bool
        +withdraw(amount: float): bool
        +get_transaction_history(): List[Transaction]
        +__str__(): str
        +__eq__(other): bool
    }

    class SavingsAccount {
        -_interest_rate: float
        -_monthly_withdrawal_count: int
        -_withdrawal_limit: int
        +withdraw(amount: float): bool
        +calculate_interest(): float
        +reset_monthly_limit(): void
    }

    class CheckingAccount {
        -_overdraft_limit: float
        -_transaction_fee: float
        +withdraw(amount: float): bool
        +get_available_balance(): float
        +charge_fee(amount: float): void
    }

    class TimeDepositAccount {
        -_maturity_date: datetime
        -_interest_rate: float
        -_penalty_rate: float
        +withdraw(amount: float): bool
        +calculate_maturity_value(): float
        +early_withdrawal_penalty(): float
    }

    class Customer {
        -_customer_id: str
        -_name: str
        -_email: str
        -_phone: str
        -_accounts: List[BankAccount]
        +customer_id: str
        +name: str
        +add_account(account: BankAccount): void
        +remove_account(account_number: str): bool
        +get_total_balance(): float
        +get_accounts_by_type(account_type: str): List[BankAccount]
    }

    class Transaction {
        -_transaction_id: str
        -_transaction_type: str
        -_amount: float
        -_timestamp: datetime
        -_description: str
        -_balance_after: float
        +__str__(): str
        +__eq__(other): bool
    }

    class Bank {
        -_bank_name: str
        -_customers: Dict[str, Customer]
        -_accounts: Dict[str, BankAccount]
        -_transaction_counter: int
        +create_customer(customer_id, name, email, phone): Customer
        +create_account(customer_id, account_type, initial_balance): BankAccount
        +find_customer(customer_id): Customer
        +find_account(account_number): BankAccount
        +transfer_funds(from_account, to_account, amount): bool
        +get_bank_statistics(): Dict
    }

    BankAccount <|-- SavingsAccount
    BankAccount <|-- CheckingAccount
    BankAccount <|-- TimeDepositAccount
    Customer ||--o{ BankAccount : owns
    BankAccount ||--o{ Transaction : has
    Bank ||--o{ Customer : manages
    Bank ||--o{ BankAccount : manages
```

### 設計模式應用

1. **工廠模式 (Factory Pattern)**：`Bank.create_account()` 根據帳戶類型動態創建對應的帳戶物件
2. **策略模式 (Strategy Pattern)**：不同帳戶類型的利息計算策略
3. **觀察者模式 (Observer Pattern)**：帳戶餘額變動通知（進階功能）
4. **單例模式 (Singleton Pattern)**：確保 Bank 系統唯一實例（進階功能）

---

## 📋 功能需求詳細規格（Functional Requirements）

### 🔥 基本需求（必須完成 - 70分）

#### 1. BankAccount 基礎類別 (15分)

**核心屬性**：
- `_account_number`: 自動生成的唯一帳戶號碼（格式：100001, 100002...）
- `_balance`: 帳戶餘額（私有屬性，只能透過 property 存取）
- `_customer`: 帳戶持有人（Customer 物件引用）
- `_transactions`: 交易記錄列表
- `_created_date`: 帳戶創建日期
- `_is_active`: 帳戶狀態（是否有效）

**核心方法**：
```python
def deposit(self, amount: float) -> bool:
    """存款操作
    
    Args:
        amount: 存款金額
    
    Returns:
        bool: 操作是否成功
    
    Raises:
        ValueError: 金額無效（≤0 或非數字）
        AccountClosedException: 帳戶已關閉
    """

def withdraw(self, amount: float) -> bool:
    """提款操作（抽象方法，子類別必須實作）
    
    Args:
        amount: 提款金額
    
    Returns:
        bool: 操作是否成功
    
    Raises:
        ValueError: 金額無效
        InsufficientFundsException: 餘額不足
        AccountClosedException: 帳戶已關閉
    """

@property
def balance(self) -> float:
    """取得帳戶餘額（唯讀屬性）"""

@property
def account_number(self) -> str:
    """取得帳戶號碼（唯讀屬性）"""
```

**特殊方法**：
- `__str__()`: 回傳友善的帳戶資訊字串
- `__repr__()`: 回傳可重建物件的字串表示
- `__eq__()`: 比較兩個帳戶是否相同（比較帳戶號碼）

#### 2. 繼承類別實作 (20分)

##### SavingsAccount（儲蓄帳戶）
**特殊屬性**：
- `_interest_rate`: 年利率（預設 1.5%）
- `_monthly_withdrawal_count`: 當月提款次數
- `_withdrawal_limit`: 每月提款次數限制（預設 6次）

**特殊行為**：
- 提款受月限制約束
- 可計算利息
- 無透支功能

```python
def withdraw(self, amount: float) -> bool:
    """覆寫提款邏輯 - 檢查月提款次數限制"""

def calculate_interest(self) -> float:
    """計算當前餘額的年利息"""

def reset_monthly_limit(self):
    """重置月提款次數（每月呼叫）"""
```

##### CheckingAccount（支票帳戶）
**特殊屬性**：
- `_overdraft_limit`: 透支額度（預設 $1000）
- `_transaction_fee`: 交易手續費（預設 $10）

**特殊行為**：
- 允許透支（在額度內）
- 每筆交易收取手續費
- 無提款次數限制

```python
def withdraw(self, amount: float) -> bool:
    """覆寫提款邏輯 - 允許透支"""

def get_available_balance(self) -> float:
    """取得可用餘額（含透支額度）"""

def charge_fee(self, amount: float):
    """扣除交易手續費"""
```

##### TimeDepositAccount（定期存款帳戶）
**特殊屬性**：
- `_maturity_date`: 到期日
- `_interest_rate`: 定存利率（較高）
- `_penalty_rate`: 提前解約罰息率

**特殊行為**：
- 到期前不可提款（或收取罰息）
- 較高的利息率
- 固定期限

#### 3. Customer 客戶類別 (10分)

**屬性**：
- `_customer_id`: 客戶編號（唯一）
- `_name`: 客戶姓名
- `_email`: 電子郵件
- `_phone`: 電話號碼
- `_accounts`: 持有的帳戶列表

**核心方法**：
```python
def add_account(self, account: BankAccount):
    """新增帳戶到客戶名下"""

def remove_account(self, account_number: str) -> bool:
    """移除指定帳戶"""

def get_total_balance(self) -> float:
    """計算所有帳戶總餘額"""

def get_accounts_by_type(self, account_type: str) -> List[BankAccount]:
    """取得指定類型的所有帳戶"""
```

#### 4. Transaction 交易記錄類別 (10分)

**屬性**：
- `_transaction_id`: 交易編號（自動生成）
- `_transaction_type`: 交易類型（'deposit', 'withdraw', 'transfer'）
- `_amount`: 交易金額
- `_timestamp`: 交易時間戳
- `_description`: 交易描述
- `_balance_after`: 交易後餘額

**方法**：
```python
def __str__(self) -> str:
    """回傳交易記錄的友善格式"""
    # 範例："2024-01-15 14:30:25 | 存款 | +$500.00 | 餘額: $1,500.00"

def __eq__(self, other) -> bool:
    """比較兩筆交易是否相同（比較交易ID）"""
```

#### 5. Bank 銀行系統類別 (15分)

**屬性**：
- `_bank_name`: 銀行名稱
- `_customers`: 客戶字典 {customer_id: Customer}
- `_accounts`: 帳戶字典 {account_number: BankAccount}
- `_transaction_counter`: 交易序號計數器

**核心方法**：
```python
def create_customer(self, customer_id: str, name: str, 
                   email: str, phone: str) -> Customer:
    """建立新客戶"""

def create_account(self, customer_id: str, account_type: str, 
                  initial_balance: float, **kwargs) -> BankAccount:
    """工廠方法：建立指定類型的帳戶
    
    Args:
        account_type: 'savings', 'checking', 'time_deposit'
        **kwargs: 各帳戶類型的特殊參數
    """

def find_customer(self, customer_id: str) -> Customer:
    """根據客戶ID查詢客戶"""

def find_account(self, account_number: str) -> BankAccount:
    """根據帳戶號碼查詢帳戶"""

def transfer_funds(self, from_account: str, to_account: str, 
                  amount: float) -> bool:
    """帳戶間轉帳"""

def get_bank_statistics(self) -> Dict:
    """取得銀行統計資訊
    
    Returns:
        {
            'total_customers': int,
            'total_accounts': int,
            'total_deposits': float,
            'accounts_by_type': Dict[str, int]
        }
    """
```

---

### 🚀 進階需求（選做 - 額外30分）

#### 1. 特殊方法與運算子重載 (10分)

**比較運算子**：
```python
# BankAccount 類別新增
def __lt__(self, other: 'BankAccount') -> bool:
    """比較帳戶餘額大小"""
    return self.balance < other.balance

def __le__(self, other: 'BankAccount') -> bool:
    return self.balance <= other.balance

def __gt__(self, other: 'BankAccount') -> bool:
    return self.balance > other.balance

def __ge__(self, other: 'BankAccount') -> bool:
    return self.balance >= other.balance

def __hash__(self):
    """讓帳戶物件可作為字典鍵值"""
    return hash(self.account_number)
```

**算術運算子重載**：
```python
def __add__(self, amount: float) -> float:
    """帳戶 + 金額 = 新餘額（模擬存款後餘額）"""
    return self.balance + amount

def __sub__(self, amount: float) -> float:
    """帳戶 - 金額 = 新餘額（模擬提款後餘額）"""
    return self.balance - amount

def __iadd__(self, amount: float):
    """帳戶 += 金額（實際執行存款）"""
    self.deposit(amount)
    return self

def __isub__(self, amount: float):
    """帳戶 -= 金額（實際執行提款）"""
    self.withdraw(amount)
    return self
```

#### 2. 進階帳戶類型 (10分)

##### CreditAccount（信用卡帳戶）
```python
class CreditAccount(BankAccount):
    """信用卡帳戶 - 允許負餘額，有信用額度限制"""
    
    def __init__(self, initial_balance: float, customer: Customer,
                 credit_limit: float = 5000.0, interest_rate: float = 0.18):
        super().__init__(initial_balance, customer)
        self._credit_limit = credit_limit
        self._interest_rate = interest_rate
        self._minimum_payment = 0.0
    
    def withdraw(self, amount: float) -> bool:
        """允許超支到信用額度"""
    
    def calculate_minimum_payment(self) -> float:
        """計算最低還款金額"""
    
    def calculate_interest_charge(self) -> float:
        """計算利息費用（針對負餘額）"""
```

##### InvestmentAccount（投資帳戶）
```python
class InvestmentAccount(BankAccount):
    """投資帳戶 - 可買賣股票，有投資組合"""
    
    def __init__(self, initial_balance: float, customer: Customer):
        super().__init__(initial_balance, customer)
        self._portfolio = {}  # {symbol: quantity}
        self._trade_history = []
    
    def buy_stock(self, symbol: str, quantity: int, price: float) -> bool:
        """買進股票"""
    
    def sell_stock(self, symbol: str, quantity: int, price: float) -> bool:
        """賣出股票"""
    
    def get_portfolio_value(self, current_prices: Dict[str, float]) -> float:
        """計算投資組合總價值"""
```

#### 3. 進階系統功能 (10分)

##### 利息自動計算系統
```python
class InterestCalculator:
    """利息計算策略類別"""
    
    @staticmethod
    def simple_interest(principal: float, rate: float, days: int) -> float:
        """簡單利息計算"""
    
    @staticmethod
    def compound_interest(principal: float, rate: float, 
                         compounds_per_year: int, years: float) -> float:
        """複利計算"""

# Bank 類別新增方法
def process_monthly_interest(self):
    """每月利息處理"""
    for account in self._accounts.values():
        if isinstance(account, SavingsAccount):
            interest = account.calculate_interest()
            account.deposit(interest)
```

##### 風險管理系統
```python
class RiskManager:
    """風險管理類別"""
    
    def __init__(self):
        self._daily_limits = {'withdraw': 10000, 'transfer': 50000}
        self._suspicious_patterns = []
    
    def check_transaction_limit(self, account: BankAccount, 
                               transaction_type: str, amount: float) -> bool:
        """檢查交易限額"""
    
    def detect_suspicious_activity(self, account: BankAccount) -> List[str]:
        """偵測可疑活動"""
    
    def freeze_account(self, account: BankAccount, reason: str):
        """凍結帳戶"""
```

---

## 🛠️ 自訂例外類別（Custom Exceptions）

建立完整的例外處理體系，提供清晰的錯誤訊息：

```python
# 基礎銀行例外
class BankException(Exception):
    """銀行系統基礎例外類別"""
    pass

# 帳戶相關例外
class AccountException(BankException):
    """帳戶操作例外"""
    pass

class InsufficientFundsException(AccountException):
    """餘額不足例外"""
    def __init__(self, available_balance: float, requested_amount: float):
        self.available_balance = available_balance
        self.requested_amount = requested_amount
        super().__init__(
            f"餘額不足：可用餘額 ${available_balance:.2f}，"
            f"要求金額 ${requested_amount:.2f}"
        )

class AccountClosedException(AccountException):
    """帳戶已關閉例外"""
    def __init__(self, account_number: str):
        super().__init__(f"帳戶 {account_number} 已關閉，無法進行交易")

class InvalidAmountException(AccountException):
    """無效金額例外"""
    def __init__(self, amount: float):
        super().__init__(f"無效的交易金額：${amount:.2f}")

class WithdrawalLimitExceededException(AccountException):
    """提款次數超限例外"""
    def __init__(self, current_count: int, limit: int):
        super().__init__(
            f"本月提款次數已達上限：{current_count}/{limit}"
        )

# 客戶相關例外
class CustomerException(BankException):
    """客戶操作例外"""
    pass

class CustomerNotFoundException(CustomerException):
    """客戶未找到例外"""
    def __init__(self, customer_id: str):
        super().__init__(f"客戶 {customer_id} 不存在")

class DuplicateCustomerException(CustomerException):
    """客戶重複例外"""
    def __init__(self, customer_id: str):
        super().__init__(f"客戶 {customer_id} 已存在")

# 轉帳相關例外
class TransferException(BankException):
    """轉帳操作例外"""
    pass

class SameAccountTransferException(TransferException):
    """同帳戶轉帳例外"""
    def __init__(self, account_number: str):
        super().__init__(f"不能向同一帳戶 {account_number} 轉帳")
```

---

## 📊 資料驗證規則（Data Validation Rules）

### 金額驗證
```python
def validate_amount(amount: float) -> float:
    """驗證交易金額
    
    Rules:
    - 必須是正數
    - 不能超過 $1,000,000（單筆限額）
    - 精確到小數點後兩位
    """
    if not isinstance(amount, (int, float)):
        raise InvalidAmountException(amount)
    
    if amount <= 0:
        raise InvalidAmountException(amount)
    
    if amount > 1_000_000:
        raise InvalidAmountException(amount)
    
    return round(amount, 2)
```

### 帳戶號碼驗證
```python
def validate_account_number(account_number: str) -> str:
    """驗證帳戶號碼格式
    
    Rules:
    - 必須是 6 位數字
    - 開頭數字 1
    - 範例：100001, 100002, ...
    """
    if not isinstance(account_number, str):
        raise ValueError("帳戶號碼必須是字串")
    
    if not account_number.isdigit() or len(account_number) != 6:
        raise ValueError("帳戶號碼必須是 6 位數字")
    
    if not account_number.startswith('1'):
        raise ValueError("帳戶號碼必須以 1 開頭")
    
    return account_number
```

### 客戶資料驗證
```python
def validate_email(email: str) -> str:
    """驗證電子郵件格式"""
    import re
    pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
    if not re.match(pattern, email):
        raise ValueError(f"無效的電子郵件格式：{email}")
    return email.lower()

def validate_phone(phone: str) -> str:
    """驗證電話號碼格式"""
    import re
    # 允許格式：09XX-XXX-XXX 或 09XXXXXXXX
    pattern = r'^09\d{2}-?\d{3}-?\d{3}$'
    if not re.match(pattern, phone):
        raise ValueError(f"無效的電話號碼格式：{phone}")
    return phone.replace('-', '')
```

---

## 🔬 測試需求與測試案例（Testing Requirements）

### 單元測試覆蓋要求
- **基本功能測試覆蓋率**：≥ 80%
- **進階功能測試覆蓋率**：≥ 60%
- **例外處理測試**：所有自訂例外都需要測試

### 關鍵測試案例

#### 1. BankAccount 基礎測試
```python
def test_account_creation():
    """測試帳戶創建"""
    # 正常創建
    # 無效初始餘額
    # 重複帳戶號碼檢查

def test_deposit():
    """測試存款功能"""
    # 正常存款
    # 無效金額（負數、零、非數字）
    # 帳戶關閉狀態

def test_withdraw():
    """測試提款功能"""
    # 正常提款
    # 餘額不足
    # 無效金額
    # 帳戶關閉狀態

def test_property_access():
    """測試屬性存取"""
    # balance 屬性唯讀
    # account_number 屬性唯讀
    # 私有屬性不可直接存取
```

#### 2. 繼承功能測試
```python
def test_savings_account():
    """測試儲蓄帳戶"""
    # 月提款次數限制
    # 利息計算
    # 月限制重置

def test_checking_account():
    """測試支票帳戶"""
    # 透支功能
    # 手續費扣除
    # 可用餘額計算

def test_time_deposit_account():
    """測試定期存款"""
    # 到期前提款限制
    # 提前解約罰息
    # 到期價值計算
```

#### 3. 系統整合測試
```python
def test_bank_operations():
    """測試銀行系統操作"""
    # 客戶創建與查詢
    # 帳戶創建（工廠模式）
    # 跨帳戶轉帳
    # 統計資訊生成

def test_customer_management():
    """測試客戶管理"""
    # 多帳戶管理
    # 總餘額計算
    # 帳戶類型篩選
```

#### 4. 邊界值與錯誤測試
```python
def test_boundary_values():
    """測試邊界值"""
    # 最小/最大交易金額
    # 透支額度邊界
    # 提款次數邊界

def test_error_handling():
    """測試錯誤處理"""
    # 所有自訂例外
    # 錯誤訊息內容
    # 錯誤恢復機制
```

### 效能測試
```python
def test_performance():
    """測試系統效能"""
    # 大量帳戶創建（1000+）
    # 大量交易處理（10000+）
    # 查詢效能（O(1) 期望）
    # 記憶體使用情況
```

---

## 📝 程式碼風格與文件要求（Code Style & Documentation）

### PEP 8 風格指引
- **命名規範**：
  - 類別名稱：PascalCase (e.g., `BankAccount`, `SavingsAccount`)
  - 方法名稱：snake_case (e.g., `deposit`, `get_total_balance`)
  - 私有屬性：底線前綴 (e.g., `_balance`, `_account_number`)
  - 常數：UPPER_CASE (e.g., `MAX_TRANSACTION_AMOUNT`)

- **程式碼格式**：
  - 行長度：≤ 88 字元
  - 縮排：4 個空格
  - 空行：類別間 2 行，方法間 1 行

### Type Hints 使用
```python
from typing import List, Dict, Optional, Union
from datetime import datetime

class BankAccount:
    def __init__(self, initial_balance: float, customer: 'Customer') -> None:
        self._balance: float = initial_balance
        self._customer: Customer = customer
        self._transactions: List[Transaction] = []
    
    def deposit(self, amount: float) -> bool:
        """存款操作"""
    
    def get_transaction_history(self) -> List[Transaction]:
        """取得交易記錄"""
    
    @property
    def balance(self) -> float:
        """帳戶餘額（唯讀）"""
```

### Docstring 文件規範
使用 Google 風格的 docstring：

```python
def transfer_funds(self, from_account: str, to_account: str, 
                  amount: float) -> bool:
    """在兩個帳戶間轉帳
    
    Args:
        from_account (str): 轉出帳戶號碼
        to_account (str): 轉入帳戶號碼
        amount (float): 轉帳金額
    
    Returns:
        bool: 轉帳是否成功
    
    Raises:
        AccountNotFoundException: 帳戶不存在
        InsufficientFundsException: 餘額不足
        SameAccountTransferException: 同帳戶轉帳
        InvalidAmountException: 無效金額
    
    Example:
        >>> bank = Bank("台灣銀行")
        >>> success = bank.transfer_funds("100001", "100002", 1000.0)
        >>> print(success)
        True
    """
```

### 註解與說明
```python
class SavingsAccount(BankAccount):
    """儲蓄帳戶類別
    
    儲蓄帳戶提供利息收入，但限制每月提款次數。
    適合長期儲蓄的客戶使用。
    
    Attributes:
        interest_rate: 年利率（預設 1.5%）
        withdrawal_limit: 每月提款次數限制（預設 6 次）
    """
    
    def withdraw(self, amount: float) -> bool:
        # 檢查月提款次數限制
        if self._monthly_withdrawal_count >= self._withdrawal_limit:
            raise WithdrawalLimitExceededException(
                self._monthly_withdrawal_count, self._withdrawal_limit
            )
        
        # 執行提款（呼叫父類別方法）
        if super().withdraw(amount):
            self._monthly_withdrawal_count += 1
            return True
        return False
```

---

## 🎯 API 設計範例（API Design Examples）

### 基本使用情境
```python
# 建立銀行系統
bank = Bank("台灣第一銀行")

# 建立客戶
customer = bank.create_customer(
    customer_id="C001",
    name="王小明",
    email="ming@example.com",
    phone="0912-345-678"
)

# 開立不同類型帳戶
savings = bank.create_account(
    customer_id="C001",
    account_type="savings",
    initial_balance=10000.0,
    interest_rate=0.02  # 2% 年利率
)

checking = bank.create_account(
    customer_id="C001",
    account_type="checking",
    initial_balance=5000.0,
    overdraft_limit=2000.0
)

# 基本交易操作
savings.deposit(1000.0)          # 存款
checking.withdraw(500.0)         # 提款
checking += 1000                 # 運算子重載存款
checking -= 200                  # 運算子重載提款

# 帳戶間轉帳
bank.transfer_funds(
    from_account=savings.account_number,
    to_account=checking.account_number,
    amount=2000.0
)

# 查詢與統計
print(f"儲蓄帳戶餘額：${savings.balance:,.2f}")
print(f"支票帳戶可用餘額：${checking.get_available_balance():,.2f}")
print(f"客戶總資產：${customer.get_total_balance():,.2f}")

# 銀行統計資訊
stats = bank.get_bank_statistics()
print(f"銀行統計：{stats}")
```

### 進階使用情境
```python
# 定期存款
from datetime import datetime, timedelta

time_deposit = bank.create_account(
    customer_id="C001",
    account_type="time_deposit",
    initial_balance=50000.0,
    interest_rate=0.035,  # 3.5% 年利率
    maturity_months=12    # 12 個月定存
)

# 信用卡帳戶（進階功能）
credit = bank.create_account(
    customer_id="C001",
    account_type="credit",
    initial_balance=0.0,
    credit_limit=30000.0
)

# 帳戶比較
accounts = [savings, checking, time_deposit]
richest_account = max(accounts)  # 使用 __gt__ 比較
poorest_account = min(accounts)  # 使用 __lt__ 比較

# 排序帳戶（按餘額）
sorted_accounts = sorted(accounts, reverse=True)

# 交易記錄查詢
for transaction in savings.get_transaction_history():
    print(transaction)  # 使用 __str__ 方法

# 利息計算與派發
interest = savings.calculate_interest()
print(f"本年度預計利息：${interest:,.2f}")
```

### 錯誤處理示範
```python
try:
    # 嘗試提款超過餘額
    savings.withdraw(50000.0)
except InsufficientFundsException as e:
    print(f"提款失敗：{e}")
    print(f"可用餘額：${e.available_balance:,.2f}")

try:
    # 嘗試超過月提款次數限制
    for i in range(10):  # 超過 6 次限制
        savings.withdraw(100.0)
except WithdrawalLimitExceededException as e:
    print(f"提款限制：{e}")

try:
    # 嘗試向同一帳戶轉帳
    bank.transfer_funds(
        from_account=savings.account_number,
        to_account=savings.account_number,
        amount=1000.0
    )
except SameAccountTransferException as e:
    print(f"轉帳錯誤：{e}")
```

---

## 📈 效能要求與優化建議（Performance Requirements）

### 效能指標
- **帳戶查詢**：O(1) 時間複雜度（使用字典索引）
- **客戶查詢**：O(1) 時間複雜度
- **交易處理**：每秒處理 ≥ 1000 筆交易
- **記憶體使用**：每個帳戶物件 ≤ 1KB

### 優化策略

#### 1. 資料結構優化
```python
class Bank:
    def __init__(self, bank_name: str):
        self._bank_name = bank_name
        # 使用字典提供 O(1) 查詢效能
        self._customers: Dict[str, Customer] = {}
        self._accounts: Dict[str, BankAccount] = {}
        # 建立索引加速查詢
        self._accounts_by_customer: Dict[str, List[str]] = {}
        self._accounts_by_type: Dict[str, List[str]] = {
            'savings': [],
            'checking': [],
            'time_deposit': [],
            'credit': []
        }
```

#### 2. 懶載入（Lazy Loading）
```python
class BankAccount:
    @property
    def transaction_history(self) -> List[Transaction]:
        """懶載入交易記錄"""
        if not hasattr(self, '_transaction_history_loaded'):
            self._load_transaction_history()
            self._transaction_history_loaded = True
        return self._transactions
```

#### 3. 記憶體優化
```python
class Transaction:
    """使用 __slots__ 減少記憶體使用"""
    __slots__ = [
        '_transaction_id', '_transaction_type', '_amount',
        '_timestamp', '_description', '_balance_after'
    ]
```

#### 4. 快取機制
```python
from functools import lru_cache

class Customer:
    @lru_cache(maxsize=1)
    def get_total_balance(self) -> float:
        """快取總餘額計算結果"""
        return sum(account.balance for account in self._accounts)
    
    def _invalidate_cache(self):
        """帳戶異動時清除快取"""
        self.get_total_balance.cache_clear()
```

---

## 📋 完成檢核清單（Completion Checklist）

### 🔥 基本需求檢核（70分）

#### BankAccount 基礎類別 (15分)
- [ ] 正確的私有屬性設計 (`_balance`, `_account_number` 等)
- [ ] Property 裝飾器實作 (`balance`, `account_number`)
- [ ] 存款方法實作 (`deposit`)
- [ ] 抽象提款方法定義 (`withdraw`)
- [ ] 交易記錄功能 (`get_transaction_history`)
- [ ] 特殊方法實作 (`__str__`, `__repr__`, `__eq__`)
- [ ] 帳戶號碼自動生成機制
- [ ] 適當的例外處理

#### 繼承類別實作 (20分)
- [ ] SavingsAccount 儲蓄帳戶
  - [ ] 利息計算功能
  - [ ] 月提款次數限制
  - [ ] 覆寫提款方法
- [ ] CheckingAccount 支票帳戶
  - [ ] 透支功能實作
  - [ ] 手續費機制
  - [ ] 可用餘額計算
- [ ] TimeDepositAccount 定期存款
  - [ ] 到期日檢查
  - [ ] 提前解約罰息
  - [ ] 到期價值計算

#### Customer 客戶類別 (10分)
- [ ] 客戶基本資料管理
- [ ] 多帳戶關聯管理
- [ ] 帳戶新增/移除功能
- [ ] 總餘額計算
- [ ] 帳戶類型篩選

#### Transaction 交易記錄 (10分)
- [ ] 完整的交易資訊記錄
- [ ] 交易ID自動生成
- [ ] 友善的字串表示
- [ ] 交易比較功能

#### Bank 銀行系統 (15分)
- [ ] 客戶管理功能（CRUD）
- [ ] 帳戶管理功能（工廠模式）
- [ ] 跨帳戶轉帳功能
- [ ] 統計資訊生成
- [ ] 查詢功能（客戶、帳戶）

### 🚀 進階需求檢核（30分）

#### 特殊方法與運算子重載 (10分)
- [ ] 比較運算子 (`__lt__`, `__gt__`, `__le__`, `__ge__`)
- [ ] 算術運算子 (`__add__`, `__sub__`, `__iadd__`, `__isub__`)
- [ ] 雜湊方法 (`__hash__`)
- [ ] 迭代器支援（選做）

#### 進階帳戶類型 (10分)
- [ ] CreditAccount 信用卡帳戶
  - [ ] 信用額度管理
  - [ ] 利息計算
  - [ ] 最低還款金額
- [ ] InvestmentAccount 投資帳戶（選做）
  - [ ] 股票交易功能
  - [ ] 投資組合管理
  - [ ] 投資損益計算

#### 進階系統功能 (10分)
- [ ] 利息自動計算系統
- [ ] 風險管理機制
- [ ] 交易限額控制
- [ ] 可疑活動偵測

### 🎯 品質檢核

#### 程式碼品質
- [ ] 遵循 PEP 8 風格指引
- [ ] 完整的 Type Hints
- [ ] 詳細的 Docstring 文件
- [ ] 適當的註解說明
- [ ] 統一的命名規範

#### 例外處理
- [ ] 完整的自訂例外類別
- [ ] 清晰的錯誤訊息
- [ ] 適當的例外處理邏輯
- [ ] 錯誤恢復機制

#### 測試覆蓋
- [ ] 基本功能測試 (≥80% 覆蓋率)
- [ ] 邊界值測試
- [ ] 例外處理測試
- [ ] 整合測試

---

**🎯 完成此檢核清單即表示您已具備企業級物件導向系統開發能力！**